# TP6

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
def euclidean_distances(X):
    """
    Computes the pairwise Euclidean distances between all points in X.
    
    Args:
        X (np.array): The input data of shape (n_samples, n_features).
        
    Returns:
        distances (np.array): Pairwise distance matrix of shape (n_samples, n_samples).
    """
    n_samples = X.shape[0]
    distances = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            distances[i, j] = np.linalg.norm(X[i] - X[j])
            distances[j, i] = distances[i, j]  # Symmetric matrix
    return distances


In [ ]:
def find_k_neighbors_by_distances(distances, k):
    """
    Finds the indices of the k-nearest neighbors for each point.
    
    Args:
        distances (np.array): Pairwise distance matrix of shape (n_samples, n_samples).
        k (int): Number of neighbors.
    
    Returns:
        neighbors (np.array): Indices of the k-nearest neighbors for each point.
    """
    neighbors = np.argsort(distances, axis=1)[:, 1:k+1]  # Exclude self
    return neighbors


In [ ]:
def compute_weights(data, neighbors_indices, alpha=0.01):
    """
    Computes the reconstruction weights for each point.
    
    Args:
        data (np.array): The input data of shape (n_samples, n_features).
        neighbors_indices (np.array): Indices of the k-nearest neighbors.
        alpha (float): Regularization term to avoid singularity.
    
    Returns:
        W (np.array): Weight matrix of shape (n_samples, k).
    """
    n_samples, n_features = data.shape
    k = neighbors_indices.shape[1]
    W = np.zeros((n_samples, k))
    
    for i in range(n_samples):
        neighbors = data[neighbors_indices[i]]
        Z = neighbors - data[i]  # Center the neighbors to the point
        C = Z @ Z.T  # Local covariance matrix
        C += alpha * np.eye(k)  # Regularization to avoid singularity
        
        w = np.linalg.solve(C, np.ones(k))  # Solve for weights
        W[i] = w / np.sum(w)  # Normalize weights to sum to 1
    
    return W


In [ ]:
def lle(x, q, k=None, alpha=0.01):
    """
    Performs Locally Linear Embedding (LLE).
    
    Args:
        x (np.array): The input data of shape (n_samples, n_features).
        q (int): The target dimensionality for the embedding.
        k (int, optional): The number of nearest neighbors (if not provided, defaults to n_features + 1).
        alpha (float, optional): Regularization term (default 0.01).
    
    Returns:
        Y (np.array): The embedding of shape (n_samples, q).
    """
    n_samples, n_features = x.shape
    
    # If k is not provided, set k = n_features + 1
    if k is None:
        k = n_features + 1
    
    # Step 1: Compute pairwise distances
    distances = euclidean_distances(x)
    
    # Step 2: Find k-nearest neighbors
    neighbors_indices = find_k_neighbors_by_distances(distances, k)
    
    # Step 3: Compute the reconstruction weights
    W = compute_weights(x, neighbors_indices, alpha)
    
    # Step 4: Compute the embedding by constructing the matrix M
    M = np.zeros((n_samples, n_samples))
    
    for i in range(n_samples):
        for j in range(k):
            M[i, neighbors_indices[i, j]] -= W[i, j]
            M[neighbors_indices[i, j], i] -= W[i, j]
        M[i, i] += 1
    
    # Step 5: Solve the eigenvalue problem
    eigvals, eigvecs = np.linalg.eigh(M)
    
    # Step 6: Return the eigenvectors corresponding to the smallest q+1 eigenvalues (excluding the first one)
    return eigvecs[:, 1:q+1]


In [ ]:
# Let's assume we have some data (n_samples, n_features)
x = np.random.rand(100, 3)  # 100 samples, 3 features

# Apply LLE to reduce the data to 2 dimensions
embedding = lle(x, q=2, k=10, alpha=0.01)
